In [1]:
#import libraries
import os
import pandas as pd
import numpy as np

In [2]:
#fixing '5E54' becoming '5.00E+54' issue
def fix_PDB_ID(df):
    for i, j in enumerate(df['PDB_ID']):
        print (j)
        if len(str(j))>4:
            if '-' in j:
                X= ''.join(j.split('-')).upper()
                df.loc[i, 'PDB_ID']= X
            else:
                if '.' in j:
                    x= str(j)
                    x1= x.split('.')[0]
                    x2= x.split('+')[1]
                    X= x1+'E'+x2
                    print (X)
                    df.loc[i, 'PDB_ID']= X
                else:
                    x= str(j)
                    x1= x.split('+')[0]
                    x2= x.split('+')[1]
                    X= x1+ x2
                    print (X)
                    df.loc[i, 'PDB_ID']= X
                    
    return (df)

In [3]:
#combining all result csvs into one
# Specify the root directory where you want to start the search
root_dir = '../data/Input/'

# List to hold DataFrames
dfs = []

# Walk through all directories and subdirectories
for dirpath, dirnames, filenames in os.walk(root_dir):
    for file in filenames:
        if file.endswith('.csv'):  # Check if the file is a CSV
            print (file)
            # Get the full file path
            file_path = os.path.join(dirpath, file)
            # Read the CSV file into a DataFrame and append it to the list
            d = pd.read_csv(file_path)
            #checking if there are two rows of columns
            unnamed_columns = [col for col in d.columns if col is None or "Unnamed" in str(col)]
            if len(unnamed_columns)>0:
                d.columns = d.iloc[0]  # Set the second row (index 1) as the header
                d = d[1:].reset_index(drop=True)
            else:
                pass
            #updating the 'Refinement Resolution' to 'Resolution'
            d.columns = d.columns.str.replace("Refinement ", "", regex=False)

            #replacing ' ' with '_' from all columns name
            d.columns = d.columns.str.replace(' ', '_') 
            d1 = d[['PDB_ID', 'Experimental_Method', 'Resolution_(Å)']].copy()
            print (d1.shape)
            print (d1.columns)
            dfs.append(d1)
            

# Concatenate all DataFrames into one
D = pd.concat(dfs, ignore_index=True)

D= fix_PDB_ID(D)
print (D.shape)

rcsb_pdb_custom_report_ba0be092afc119988ec6c834c6ec500b_7501-8748.csv
(25586, 3)
Index(['PDB_ID', 'Experimental_Method', 'Resolution_(Å)'], dtype='object', name=0)
rcsb_pdb_custom_report_ba0be092afc119988ec6c834c6ec500b_5001-7500.csv
(57731, 3)
Index(['PDB_ID', 'Experimental_Method', 'Resolution_(Å)'], dtype='object', name=0)
rcsb_pdb_custom_report_ba0be092afc119988ec6c834c6ec500b_0001-2500.csv
(13947, 3)
Index(['PDB_ID', 'Experimental_Method', 'Resolution_(Å)'], dtype='object', name=0)
rcsb_pdb_custom_report_ba0be092afc119988ec6c834c6ec500b_2501-5000.csv
(53177, 3)
Index(['PDB_ID', 'Experimental_Method', 'Resolution_(Å)'], dtype='object', name=0)
8PSO
nan
nan
nan
nan
8PSQ
nan
nan
nan
8PSS
nan
nan
nan
8PSU
nan
nan
nan
8PSX
nan
nan
nan
nan
8PSZ
nan
nan
nan
nan
8PT2
nan
nan
nan
8PT6
nan
nan
nan
nan
8PT7
nan
nan
nan
8PTG
nan
8PTH
nan
nan
nan
8PTJ
nan
nan
nan
8PTX
nan
nan
nan
8PTZ
nan
nan
nan
8PU0
nan
nan
nan
8PV1
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
